In [1]:
import cv2 as cv
import numpy as np

In [2]:
from scipy.spatial.transform import Rotation
from algs.uniform_sampling import UniformSampling
from manipulated_object import ObjectPosition


def uniform_dataset(n:int, start_pos:tuple[int,int,int]):
    axes = UniformSampling._uniform_det_axes(n**2)
    axes = np.repeat(axes, n, axis=0)
    rots = np.linspace(0, 2 * np.pi, num=n, endpoint=False)
    rots = np.tile(rots, n**2)
    rot_vec = np.expand_dims(rots, axis=-1) * axes
    orients = Rotation.from_rotvec(rot_vec).as_euler("xyz")
    
    dataset = [ObjectPosition(orient, start_pos) for orient in orients]
    return dataset

In [5]:
from view_sampler import ViewSampler, CameraConfig
from manipulated_object import ObjectPosition
import loss_funcs
from evaluate import eval_funcs
from algs import *

from evaluate.evaluator import Evaluator
from visualize import SearchPlotter
from algs.mealpy_algorithm import MealpyAlgorithm
from mealpy.physics_based.CDO import OriginalCDO
from mealpy.swarm_based.PSO import OriginalPSO

# Create a camera configuration
cam_config = CameraConfig(location=(0, 0, 0.1), rotation=(np.pi / 2, 0, 0), fov=60)
world_viewer = ViewSampler("data/world_mug.xml", cam_config, simulation_time=5)
sim_viewer = ViewSampler("data/world_mug_sim.xml", cam_config)

experiment_folder = "Evals/test2"

In [6]:
init_location = (0, 1.3, 0.3)
random_orientations = np.random.uniform(0, 2 * np.pi, size=(5, 3))
# eval_positions = [ObjectPosition(orient, init_location) for orient in random_orientations]
eval_positions = uniform_dataset(10, init_location)


evaluator = Evaluator(world_viewer, eval_func=eval_funcs.NormXorDiff(obj_depth=1.0, method="mae"))
evaluator.enable_logging(experiment_folder)

In [7]:
# plotter = SearchPlotter()

loss_func = loss_funcs.IOU()

alg = MealpyAlgorithm(sim_viewer, loss_func=loss_func, optimizer=OriginalCDO())
alg_config = MealpyAlgorithm.Config(time_limit=10, seed=1)


for alg, config in [(alg, alg_config)]:
    eval_losses = evaluator.evaluate(alg, config, eval_positions)
    print(f"{type(alg).__name__}: {eval_losses}")

# plotter.close()
world_viewer.close()

Evaluating Algorithm: MealpyAlgorithm
Alg Config: MealpyAlgorithm.Config(time_limit=10, seed=1, run_mode='single', n_workers=None, log_dest='console')
Loss Function: IOU


Evaluating:   0%|          | 0/5 [00:00<?, ?it/s]

2024/03/16 06:38:10 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: Solving single objective optimization problem.
/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/manipulated_object.py:27: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  rotation = Rotation.from_quat(quat).as_euler("xyz", degrees=False)
2024/03/16 06:38:11 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 1, Current best: 0.11671732522796352, Global best: 0.11671732522796352, Runtime: 0.49427 seconds
2024/03/16 06:38:12 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 2, Current best: 0.11625076080340835, Global best: 0.11625076080340835, Runtime: 0.46854 seconds
2024/03/16 06:38:12 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 3, Current best: 0.11449451887941531, Global best: 0.11449451887941531,

2024/03/16 06:38:21 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 1, Current best: 0.22116603295310522, Global best: 0.22116603295310522, Runtime: 0.35815 seconds
2024/03/16 06:38:22 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 2, Current best: 0.2136164896939413, Global best: 0.2136164896939413, Runtime: 0.34195 seconds
2024/03/16 06:38:22 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 3, Current best: 0.2209595959595959, Global best: 0.2136164896939413, Runtime: 0.36843 seconds
2024/03/16 06:38:22 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 4, Current best: 0.21860767729342878, Global best: 0.2136164896939413, Runtime: 0.40592 seconds
2024/03/16 06:38:23 PM, INFO, mealpy.physics_based.CDO.OriginalCDO: >>>Problem: orient_detection, Epoch: 5, Current best: 0.2173076923076923, Global best: 0.2136164896939413, Runtime: 0.35364 secon

MealpyAlgorithm: [0.5524484498014353, 0.6856942031499531, 0.47790674659146726, 0.728911145196911, 0.5939874699056923]


In [8]:
from exp_logging import Experiment

exp_path = "/Users/guycohen/Library/CloudStorage/OneDrive-Technion/1 Courses/Robotics/Robotics-Proj/Evals/test2/OriginalCDO.res"

exp = Experiment.load(exp_path)



In [11]:
exp.history_list[0].__dict__

{'list_global_best': [id: 212, target: Objectives: [0.11671733], Fitness: 0.11671732522796352, solution: [1.09171373 1.57079633 3.14159265],
  id: 318, target: Objectives: [0.11625076], Fitness: 0.11625076080340835, solution: [1.09727385 1.57079633 3.14159265],
  id: 424, target: Objectives: [0.11449452], Fitness: 0.11449451887941531, solution: [1.10407272 1.57079633 3.14159265],
  id: 530, target: Objectives: [0.11449452], Fitness: 0.11449451887941531, solution: [1.10407272 1.57079633 3.14159265],
  id: 636, target: Objectives: [0.11449452], Fitness: 0.11449451887941531, solution: [1.10407272 1.57079633 3.14159265],
  id: 742, target: Objectives: [0.11449452], Fitness: 0.11449451887941531, solution: [1.10407272 1.57079633 3.14159265],
  id: 848, target: Objectives: [0.11449452], Fitness: 0.11449451887941531, solution: [1.10407272 1.57079633 3.14159265],
  id: 954, target: Objectives: [0.11449452], Fitness: 0.11449451887941531, solution: [1.10407272 1.57079633 3.14159265],
  id: 1060, 